Imports

In [205]:
import feed_forward
import feed_forward_EUCL
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
torch.cuda.is_available()

False

CUDA check

In [206]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [207]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
                                ])

In [208]:
trainset = datasets.CIFAR10('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.CIFAR10('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [209]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in trainset if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in valset if label in [0, 2]]

trainloader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

Initializing the model

In [210]:
model = feed_forward.HypFF(3072, 1024, 512, 2, nn.ReLU())
ball = geoopt.PoincareBall()
model

HypFF(
  (act_fn): ReLU()
  (fc1): MobLinear(
    in_features=3072, out_features=1024, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=1024, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc3): MobLinear(
    in_features=512, out_features=2, bias=True
    (ball): PoincareBall manifold
  )
)

In [211]:
learning_rate = 4e-1
momentum = 0.9

In [212]:
optimizer = geoopt.optim.RiemannianSGD(model.parameters(), 
                                       lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
loss_fn = nn.CrossEntropyLoss()

In [213]:
n_epochs = 15
for epoch in range(n_epochs):
    for imgs, labels in trainloader:
        # img, label = img.to(device), label.to(device)
        batch_size = imgs.shape[0]
        out = model(ball.projx(imgs.view(batch_size, -1)))
        loss = loss_fn(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.694988
Epoch: 1, Loss: 0.692429
Epoch: 2, Loss: 0.693146
Epoch: 3, Loss: 0.684861
Epoch: 4, Loss: 0.688185
Epoch: 5, Loss: 0.700493
Epoch: 6, Loss: 0.694231
Epoch: 7, Loss: 0.689970
Epoch: 8, Loss: 0.711954
Epoch: 9, Loss: 0.691582
Epoch: 10, Loss: 0.693056
Epoch: 11, Loss: 0.704615
Epoch: 12, Loss: 0.689394
Epoch: 13, Loss: 0.693639


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in valloader:
        batch_size = imgs.shape[0]
        outputs = model(ball.projx(imgs.view(batch_size, -1)))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f", correct / total)

Accuracy: %f 0.859
